In [1]:
#need 2 go through each month_year, 
#change those points to shapefile
#run rest of script on them
#preserve those hexes that consistantly have 5+ fires month-over-month 
#write hex id with #of fires, & month_year

import shapefile, shapely, rtree, csv, os, sys
from collections import Counter
from shapely.geometry import Point
from shapely.geometry import Polygon
from rtree import index
import pandas as pd

#script to count up how many points fall within each hexigon. outputs csv you can attribute join back to hexigon feature class

polyfeat = r"/Users/itclunie/Desktop/ECON/find industry/hexes/grid4.shp"        #hex polygons
pointfeat = r"/Users/itclunie/Desktop/ECON/find industry/VIIRScluster_points/VIIRSclustRd.shp"     #points

In [2]:
#A Load the shapefile of polygons and convert it to shapely polygon objects
polygons_sf = shapefile.Reader(polyfeat)
polygon_shapes = polygons_sf.shapes()
polygon_points = [q.points for q in polygon_shapes ]
polygons = [Polygon(q) for q in polygon_points]
poly_records = polygons_sf.records()

In [3]:
#B Load the shapefile of points and convert it to shapely point objects
points_sf = shapefile.Reader(pointfeat)

pntRecords = points_sf.shapeRecords()

point_coords = [q.shape.points[0] for q in pntRecords ]
points = [Point(q.shape.points[0]) for q in pntRecords ]

In [6]:
#C structure tally dictionaries
HEXdict = {}
subHEXdict = {}
VIIRSdict = {}
for i in pntRecords:
    VIIRSdict[i.record[4]] = [] #add keys (monthYear) to VIIRSdict & HEXdict
    HEXdict[i.record[4]] = []
    
for i in pntRecords: #fill VIIRSdict.  
    VIIRSdict[i.record[4]].append(i) #key: monthYear, value: shape and attributes

    
for i in poly_records: #fill subHEXdict.  
    subHEXdict['hex_' + str(i[0]) ] = 0 ##key= hexID, value= {hexid:0...    looks like {0:0, 1:0, 2:0 ...

for key in HEXdict:  #fill HEXdict. 
    HEXdict[key] = dict(subHEXdict)  #key= 8_2012  value=  copies of subHEXdict    
    


In [7]:
#D Build a spatial index based on the bounding boxes of the polygons
idx = index.Index()
count = -1
for q in polygon_shapes:
    count +=1
    idx.insert(count, q.bbox)

In [8]:
#E Assign one or more matching polygons to each point
for key in VIIRSdict:
    print(key)
    
    tallyHO = []
    point_coords = [ q.shape.points[0] for q in VIIRSdict[key] ]
    points = [ Point(q.shape.points[0]) for q in VIIRSdict[key] ]    
    
    for i in range(len(VIIRSdict[key])): #Iterate through each point

        #Iterate only through the bounding boxes which contain the point 
        for j in idx.intersection(point_coords[i]):
            
            #Verify that point is within the polygon itself not just the bounding box
            if points[i].within(polygons[j]):       
                tallyHO.append(poly_records[j][0]) 
                break 
                
    resultDict = dict([ (i,tallyHO.count(i)) for i in set(tallyHO) ])

    for rkey in resultDict:
        HEXdict[key][rkey] = resultDict[rkey]          
    

8_2017
7_2017
6_2017
3_2017
2_2017
1_2017
12_2016
11_2016
10_2016
9_2016
8_2016
7_2016
6_2016
5_2016
3_2016
2_2016
1_2016
12_2015
11_2015
10_2015
9_2015
8_2015
7_2015
6_2015
5_2015
3_2015
1_2015
11_2014
10_2014
9_2014
8_2014
7_2014
6_2014
3_2014
2_2014
1_2014
12_2013
11_2013
10_2013
9_2013
8_2013
7_2013
6_2013
5_2013
3_2013
1_2013
12_2012
11_2012
10_2012
9_2012
8_2012
7_2012
6_2012
2_2012
1_2018
12_2017
11_2017
10_2017
9_2017
5_2017
4_2016
4_2017
4_2015
2_2015
5_2014
4_2014
4_2013
5_2012
4_2012
3_2012
12_2014
2_2013
2_2018


In [17]:
df = pd.DataFrame.from_dict(HEXdict, orient='columns', dtype=None)
df['hexid'] = df.index

#melted = df.melt(id_vars=['hexid']) # .melt(df, id_vars=["weekday"], var_name="Person", value_name="Score")
#melted.to_csv('test.csv') 

df.head()

,10_2012,10_2013,10_2014,10_2015,10_2016,10_2017,11_2012,11_2013,11_2014,11_2015,...,8_2015,8_2016,8_2017,9_2012,9_2013,9_2014,9_2015,9_2016,9_2017,hexid
hex_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hex_0
hex_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hex_1
hex_10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hex_10
hex_100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hex_100
hex_1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hex_1000
